# Digest minute crypto data (prices) and coinmarketcap (market)

- https://www.bitfinex.com/
- https://medium.com/coinmonks/how-to-get-historical-crypto-currency-data-954062d40d2d
- https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution


In [1]:
%matplotlib inline

from pathlib import Path
import pandas as pd

idx = pd.IndexSlice

In [2]:

# https://stackoverflow.com/questions/16466670/fill-nan-in-candlestick-ohlcv-data
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html
def nans_to_prev_close_method1(df):
    df['volume'] = df['volume'].fillna(0.0)  # volume should always be 0 (if there were no trades in this interval)
    df['close'] = df.fillna(method='pad')  # ie pull the last close into this close
    # now copy the close that was pulled down from the last timestep into this row, across into o/h/l
    df['open'] = df['open'].fillna(df['close']) 
    df['low'] = df['low'].fillna(df['close'])
    df['high'] = df['high'].fillna(df['close'])
    # print(df.head(10))
    return df

# 
def resample_df_ohlcv(df, timeframe='15Min'):
    # Created a dictionary to tell Pandas how to re-sample, if this isn't in place it will re-sample each column separately
    ohlcv_dict = {'open':'first', 'high':'max', 'low':'min', 'close': 'last', 'volume': 'sum'}

    # Resample to 15Min (this format is needed) as per ohlc_dict, then remove any line with a NaN
    df = df.resample(timeframe).agg(ohlcv_dict)
    df = nans_to_prev_close_method1(df)
    # Resample mixes the columns so lets re-arrange them
    cols=['open', 'high', 'low', 'close', 'volume']  
    df = df[cols]
    return df




In [3]:
data_path ='../data/crypto_1min/'

cryptos = {
            'btcusd':  {'symbol': 'btc', 'base': 'usd', },
            'ethusd':  {'symbol': 'eth', 'base': 'usd', },
            # Binance Coin
            'xrpusd':  {'symbol': 'xrp', 'base': 'usd', },
            'ustusd':  {'symbol': 'usdt', 'base': 'usd', },
            'adausd':  {'symbol': 'ada', 'base': 'usd', }, # Cardano
            'dogusd':  {'symbol': 'doge', 'base': 'usd',},
            'dotusd':  {'symbol': 'dot', 'base': 'usd', }, # Polkadot
            'uniusd':  {'symbol': 'uni', 'base': 'usd', },
            'ltcusd':  {'symbol': 'ltc', 'base': 'usd', },
            'bchnusd':  {'symbol': 'bch', 'base': 'usd', }, # Bitcoin Cash ? bch = bchn? 
            'linkusd':  {'symbol': 'link', 'base': 'usd', }, # Chainlink
            'udcusd':  {'symbol': 'usdc', 'base': 'usd', },   # USDC
            'vetusd':  {'symbol': 'vet', 'base': 'usd', }, # VeChain
            'solusd':  {'symbol': 'sol', 'base': 'usd',}, # Solana
            'xlmusd':  {'symbol': 'xlm', 'base': 'usd',}, # Stellar
            # 'theta', 
            'filusd':  {'symbol': 'fil', 'base': 'usd',}, # filecoin
            'okbusd': {'symbol': 'okb', 'base': 'usd'},
            'wbtusd': {'symbol': 'wbtc', 'base': 'usd'},

            'trxusd':  {'symbol': 'trx', 'base': 'usd',}, # Tron
            # Binance USD busd
            
            'bsvusd': {'symbol': 'bsv', 'base': 'usd'},

            'xmrusd':  {'symbol': 'xmr', 'base': 'usd',}, # Monero
            'lunausd':  {'symbol': 'luna', 'base': 'usd',}, # Luna
            'neousd':  {'symbol': 'neo', 'base': 'usd',}, # 
            # Klay
            'iotusd':  {'symbol': 'miota', 'base': 'usd',}, # iota 
            'eosusd':  {'symbol': 'eos', 'base': 'usd',}, # iota 
            'atousd':  {'symbol': 'atom', 'base': 'usd',}, # cosmos 
            # 'cakusd':  {'symbol': 'cake', 'base': 'usd',}, # pancake 
            'aaveusd':  {'symbol': 'aave', 'base': 'usd',}, # aave
            'eosusd': {'symbol': 'eos', 'base': 'usd'},
            'etcusd': {'symbol': 'etc', 'base': 'usd'},
            'atousd': {'symbol': 'atom', 'base': 'usd'},
            # 'maticusd': {'symbol': '...', 'base': 'usd'},
            # ht
            'fttusd': {'symbol': 'ftt', 'base': 'usd'},
            # cro
            'bttusd': {'symbol': 'btt', 'base': 'usd'},
            # cusd
            'mkrusd': {'symbol': 'mkr', 'base': 'usd'},
            # 'mkreth': {'symbol': 'mkr', 'base': 'eth'},
            # 'mkrbtc': {'symbol': 'mkr', 'base': 'btc'},
            'xtzusd': {'symbol': 'xtz', 'base': 'usd'},
            # 'xtzbtc': {'symbol': 'xtz', 'base': 'btc'},
            # ceth
            'algusd': {'symbol': 'algo', 'base': 'usd'},
            'avaxusd': {'symbol': 'avax', 'base': 'usd'},
            'ksmusd': {'symbol': 'ksm', 'base': 'usd'},
            'daiusd': {'symbol': 'dai', 'base': 'usd'},
            # cdai
            # dash
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            # '...usd': {'symbol': '...', 'base': 'usd'},
            'mtnusd': {'symbol': 'mtn', 'base': 'usd'},
            'manusd': {'symbol': 'man', 'base': 'usd'},

            }


<bound method IndexOpsMixin.tolist of Index(['btc', 'eth', 'bnb', 'xrp', 'usdt', 'doge', 'ada', 'dot', 'uni', 'bch',
       'ltc', 'link', 'usdc', 'vet', 'sol', 'xlm', 'theta', 'fil', 'okb','wbtc', 'trx', 'busd',
       'xmr', 'neo', 'luna', 'cake', 'bsv', 'aave', 'eos', 'klay', 'miota', 'etc', 'atom', 
       
       'matic', 'ht', 'ftt', 'cro',
       'btt', 'cusdc', 'mkr', 'comp', 'xtz', 'ceth', 'algo', 'AVAX', 'ksm',
       'dai',
       'cdai', 'rune', 'dash', 'xem', 'egld', 'chz', 'hot', 'zec',
       'hbar', 'dcr', 'snx', 'enj', 'zil', 'waves', 'cel', 'leo', 'dgb',
       'sushi', 'stx', 'amp', 'nexo', 'sc', 'ftm', 'grt', 'ust', 'near',
       'mana', 'bat', 'yfi', 'rvn', 'btg', 'icx', 'qtum', 'hbtc', 'uma', 'hnt',
       'zrx', 'lusd', 'nano', 'ont', 'iost', 'one', 'zen', 'bnt', 'chsb',
       'arrr', 'ankr', 'ar', 'xvs', 'pax', 'flow', 'bake', 'kcs'],
      dtype='object', name='symbol')>



Load minute data 

In [4]:
def three_spaces():
    print("")
    print("")
    print("")

def data_from_folder(csv_path, cryptos):
    
    prices = []
    for symbol_key, symbol_dict in cryptos.items():
        file_path = data_path + symbol_key + ".csv"
        file_path = Path(file_path)
        df = pd.read_csv(file_path) # read the csv 
        df['date'] = df['time'].values.astype(dtype='datetime64[ms]') # for msec format   
        df.drop(df.filter(regex="time"),axis=1, inplace=True) # otherwise time row is left behind as "time"
        df.set_index(['date'], inplace=True)

        # resample to desired frequency
        df = resample_df_ohlcv(df, '60Min')
        df['symbol'] = symbol_dict['symbol'] # get a symbol column
        df['base'] = symbol_dict['base'] # get a base column
        df.set_index(['symbol', 'base'], inplace=True, append=True)

        df.sort_index(inplace=True)
        df.drop(df.filter(regex="Unname"),axis=1, inplace=True) # otherwise index row is left behind as "unnamed"
        prices.append(df)
    prices = pd.concat(prices)
    
    prices.sort_index(inplace=True)
    print("found {} symbol(s):\n {}".format(len(prices.index.get_level_values('symbol').unique()),
                                          prices.index.get_level_values('symbol').unique()))
    print("found {} base(s):\n {}".format(len(prices.index.get_level_values('base').unique()),
                                       prices.index.get_level_values('base').unique()))
    return prices


prices = data_from_folder(data_path, cryptos)
three_spaces()
print('RESULTING DF: ')
print(prices.info())

print(prices.index.get_level_values('symbol').unique())

found 38 symbol(s):
 Index(['btc', 'ltc', 'eth', 'etc', 'xmr', 'xrp', 'miota', 'eos', 'neo', 'trx',
       'dai', 'mtn', 'xlm', 'mkr', 'man', 'vet', 'xtz', 'bsv', 'usdt', 'usdc',
       'btt', 'atom', 'wbtc', 'okb', 'algo', 'ftt', 'doge', 'ada', 'dot',
       'ksm', 'uni', 'fil', 'sol', 'aave', 'avax', 'bch', 'link', 'luna'],
      dtype='object', name='symbol')
found 1 base(s):
 Index(['usd'], dtype='object', name='base')



RESULTING DF: 
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 811703 entries, (Timestamp('2013-04-01 00:00:00'), 'btc', 'usd') to (Timestamp('2021-05-13 06:00:00'), 'xtz', 'usd')
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    811703 non-null  float64
 1   high    811703 non-null  float64
 2   low     811703 non-null  float64
 3   close   811703 non-null  float64
 4   volume  811703 non-null  float64
dtypes: float64(5)
memory usage: 38.2+ MB
None
Index(['btc', 'ltc', 'eth', 'etc', 'xmr', '

In [5]:
prices.head(10)

,,,open,high,low,close,volume
date,symbol,base,,,,,
2013-04-01 00:00:00,btc,usd,93.25000,100.00000,93.03000,93.25000,345.583889
2013-04-01 01:00:00,btc,usd,93.37999,93.48797,93.10000,93.37999,45.243335
2013-04-01 02:00:00,btc,usd,93.17000,94.00000,93.10999,93.17000,466.311420
2013-04-01 03:00:00,btc,usd,93.80000,93.80000,92.49999,93.80000,96.316180
2013-04-01 04:00:00,btc,usd,93.15501,93.15503,93.15501,93.15501,5.000000
2013-04-01 05:00:00,btc,usd,93.70000,93.79000,93.70000,93.70000,22.090000
2013-04-01 06:00:00,btc,usd,94.06751,94.48000,94.00000,94.06751,84.000000
2013-04-01 07:00:00,btc,usd,94.06751,94.06751,94.06751,94.06751,0.000000
2013-04-01 08:00:00,btc,usd,93.55000,94.00000,93.55000,93.55000,3.807371


In [6]:
prices.tail(10)

open         high          low  \
date                symbol base                                          
2021-05-13 06:00:00 trx    usd       0.12517      0.12623      0.12517   
                    uni    usd      39.65200     39.93300     39.60200   
                    usdc   usd       0.99990      1.00040      0.99960   
                    usdt   usd       1.00060      1.00070      1.00010   
                    vet    usd       0.18527      0.18921      0.18499   
                    wbtc   usd   50912.00000  51320.00000  50912.00000   
                    xlm    usd       0.62706      0.63696      0.62446   
                    xmr    usd     417.02000    419.45000    417.02000   
                    xrp    usd       1.38470      1.38770      1.37720   
                    xtz    usd       6.30770      6.38470      6.30290   

                                       close        volume  
date                symbol base                             
2021-05-13 06:00:00 trx    usd       0.12517  3.188056e+05  
                    uni    usd      39.65200  8.903556e+03  
                    usdc   usd       0.99990  1.501171e+04  
                    usdt   usd       1.00060  4.270349e+06  
                    vet    usd       0.18527  1.801627e+06  
                    wbtc   usd   50912.00000  1.189760e-03  
                    xlm    usd       0.62706  3.017349e+05  
                    xmr    usd     417.02000  6.087661e+01  
                    xrp    usd       1.38470  3.860833e+05  
                    xtz    usd       6.30770  4.503583e+04

In [7]:
DATA_STORE = '../data/crypto.h5'
with pd.HDFStore(DATA_STORE) as store:
    store.put('crypto/caggle/prices', prices)
print("done")


done


https://zaxrosenberg.com/pandas-multiindex-tutorial/


In [8]:
prices.index.get_level_values('symbol').unique()

Index(['btc', 'ltc', 'eth', 'etc', 'xmr', 'xrp', 'miota', 'eos', 'neo', 'trx',
       'dai', 'mtn', 'xlm', 'mkr', 'man', 'vet', 'xtz', 'bsv', 'usdt', 'usdc',
       'btt', 'atom', 'wbtc', 'okb', 'algo', 'ftt', 'doge', 'ada', 'dot',
       'ksm', 'uni', 'fil', 'sol', 'aave', 'avax', 'bch', 'link', 'luna'],
      dtype='object', name='symbol')

In [9]:
# inplace
idx = pd.IndexSlice
prices.loc[idx['2021']]

open          high          low  \
date                symbol base                                           
2021-01-01 00:00:00 ada    usd       0.18110      0.181100      0.17850   
                    algo   usd       0.33618      0.346470      0.33147   
                    atom   usd       6.44000      6.440000      6.27230   
                    bsv    usd     163.28000    163.350000    161.94000   
                    btc    usd   28939.00000  29038.426155  28716.00000   
...                                      ...           ...          ...   
2021-05-13 06:00:00 wbtc   usd   50912.00000  51320.000000  50912.00000   
                    xlm    usd       0.62706      0.636960      0.62446   
                    xmr    usd     417.02000    419.450000    417.02000   
                    xrp    usd       1.38470      1.387700      1.37720   
                    xtz    usd       6.30770      6.384700      6.30290   

                                       close         volume  
date                symbol base                              
2021-01-01 00:00:00 ada    usd       0.18110   76856.980277  
                    algo   usd       0.33618   65817.281904  
                    atom   usd       6.44000   10823.682004  
                    bsv    usd     163.28000      30.913340  
                    btc    usd   28939.00000     316.524709  
...                                      ...            ...  
2021-05-13 06:00:00 wbtc   usd   50912.00000       0.001190  
                    xlm    usd       0.62706  301734.858587  
                    xmr    usd     417.02000      60.876606  
                    xrp    usd       1.38470  386083.274679  
                    xtz    usd       6.30770   45035.830311  

[100127 rows x 5 columns]

In [10]:
# copies.. 
prices.xs('btc', level='symbol').head(10)

,,open,high,low,close,volume
date,base,,,,,
2013-04-01 00:00:00,usd,93.25000,100.00000,93.03000,93.25000,345.583889
2013-04-01 01:00:00,usd,93.37999,93.48797,93.10000,93.37999,45.243335
2013-04-01 02:00:00,usd,93.17000,94.00000,93.10999,93.17000,466.311420
2013-04-01 03:00:00,usd,93.80000,93.80000,92.49999,93.80000,96.316180
2013-04-01 04:00:00,usd,93.15501,93.15503,93.15501,93.15501,5.000000
2013-04-01 05:00:00,usd,93.70000,93.79000,93.70000,93.70000,22.090000
2013-04-01 06:00:00,usd,94.06751,94.48000,94.00000,94.06751,84.000000
2013-04-01 07:00:00,usd,94.06751,94.06751,94.06751,94.06751,0.000000
2013-04-01 08:00:00,usd,93.55000,94.00000,93.55000,93.55000,3.807371


In [11]:
# buggy date
prices.xs(('2018-07-01', 'btc', 'usd'), level=['date', 'symbol', 'base'])


,,,open,high,low,close,volume
date,symbol,base,,,,,
2015-12-09 20:00:00,btc,usd,414.11,414.56,412.56,414.11,436.156913
2015-12-09 21:00:00,btc,usd,414.51,417.25,413.68,414.51,1172.483208
2015-12-09 22:00:00,btc,usd,417.17,418.45,417.00,417.17,709.583719
2015-12-09 23:00:00,btc,usd,417.75,419.45,416.50,417.75,1081.349244
2015-12-10 00:00:00,btc,usd,418.86,421.00,414.90,418.86,1963.019398
2015-12-10 01:00:00,btc,usd,416.00,418.71,415.97,416.00,713.827955
2015-12-10 02:00:00,btc,usd,417.19,417.77,413.78,417.19,705.689091
2015-12-10 03:00:00,btc,usd,417.74,417.98,413.33,417.74,1031.438780
2015-12-10 04:00:00,btc,usd,415.69,416.17,414.30,415.69,543.623548


In [12]:
prices.xs(('btc', 'usd'), level=['symbol', 'base'])

,open,high,low,close,volume
date,,,,,
2013-04-01 00:00:00,93.250000,100.00000,93.030000,93.250000,345.583889
2013-04-01 01:00:00,93.379990,93.48797,93.100000,93.379990,45.243335
2013-04-01 02:00:00,93.170000,94.00000,93.109990,93.170000,466.311420
2013-04-01 03:00:00,93.800000,93.80000,92.499990,93.800000,96.316180
2013-04-01 04:00:00,93.155010,93.15503,93.155010,93.155010,5.000000
...,...,...,...,...,...
2021-05-13 02:00:00,50517.003812,50879.00000,50066.000000,50517.003812,1093.080290
2021-05-13 03:00:00,50312.000000,50501.00000,49585.409306,50312.000000,909.961647
2021-05-13 04:00:00,50481.000000,51008.00000,50315.000000,50481.000000,781.345590


In [13]:

# seems only the first result
result = prices.query("'symbol =='btc'")
print(result)

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
'btc' in prices.index.get_level_values('symbol') and 'usd' in prices.index.get_level_values('base')

In [ ]:
'btc' in prices.index.get_level_values('symbol') and 'xxxn' in prices.index.get_level_values('base')